In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/job-recommendation/window_dates.tsv
/kaggle/input/job-recommendation/apps.zip
/kaggle/input/job-recommendation/users.tsv
/kaggle/input/job-recommendation/apps.tsv
/kaggle/input/job-recommendation/users.zip
/kaggle/input/job-recommendation/splitjobs.zip
/kaggle/input/job-recommendation/popular_jobs.csv
/kaggle/input/job-recommendation/user_history.zip
/kaggle/input/job-recommendation/user_history.tsv
/kaggle/input/job-recommendation/test_users.tsv
/kaggle/input/job-recommendation/test_users.zip
/kaggle/input/job-recommendation/popular_jobs.py
/kaggle/input/job-recommendation/jobs.zip


In [2]:
!mkdir /kaggle/temp
!unzip ../input/job-recommendation/jobs.zip
!mv  /kaggle/working/jobs.tsv /kaggle/temp/

Archive:  ../input/job-recommendation/jobs.zip
  inflating: jobs.tsv                


In [3]:
apps = pd.read_csv('/kaggle/input/job-recommendation/apps.tsv', delimiter='\t',encoding='utf-8')
user_history = pd.read_csv('/kaggle/input/job-recommendation/user_history.tsv', delimiter='\t',encoding='utf-8')
jobs = pd.read_csv('/kaggle/temp/jobs.tsv', delimiter='\t',encoding='utf-8', nrows=99999)
users = pd.read_csv('/kaggle/input/job-recommendation/users.tsv' ,delimiter='\t',encoding='utf-8')
test_users = pd.read_csv('/kaggle/input/job-recommendation/test_users.tsv', delimiter='\t',encoding='utf-8')

/tmp/ipykernel_32/433416094.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv('/kaggle/temp/jobs.tsv', delimiter='\t',encoding='utf-8', nrows=99999)


In [4]:
user_history.head()

,UserID,WindowID,Split,Sequence,JobTitle
0,47,1,Train,1,National Space Communication Programs-Special ...
1,47,1,Train,2,Detention Officer
2,47,1,Train,3,"Passenger Screener, TSA"
3,72,1,Train,1,"Lecturer, Department of Anthropology"
4,72,1,Train,2,Student Assistant


In [4]:
user_history_training = user_history.loc[user_history['Split'] =='Train']
user_history_testing = user_history.loc[user_history['Split'] =='Test']
apps_training = apps.loc[apps['Split'] == 'Train']
apps_testing = apps.loc[apps['Split'] == 'Test']
users_training = users.loc[users['Split']=='Train']
users_testing = users.loc[users['Split']=='Test']

In [5]:
user_history_training = user_history_training[user_history_training['UserID'] < 50000]
user_history_testing = user_history_testing[user_history_testing['UserID'] < 50000]
apps_training = apps_training[apps_training['UserID'] < 50000]
apps_testing = apps_testing[apps_testing['UserID'] < 50000]
users_training = users_training[users_training['UserID'] < 50000]
users_testing = users_testing[users_testing['UserID'] < 50000]

In [6]:
test_users = test_users[test_users['UserID'] < 50000]
users_training = users_training.loc[users_training['Country']=='US']
users_testing = users_testing.loc[users_testing['Country']=='US']
jobs_US = jobs.loc[jobs['Country']=='US']

In [7]:
common_user_ids = set(users_training['UserID']).intersection(apps_training['UserID'])
filtered_users_training = users_training[users_training['UserID'].isin(common_user_ids)]
filtered_apps_training = apps_training[apps_training['UserID'].isin(common_user_ids)]

## Applicants-based collaborative filtering model

In [11]:
filtered_users_training.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0
5,131,1,Train,Houston,TX,US,77077,Bachelor's,Finance,1998-05-01 00:00:00,3,14.0,NaN,No,0


In [8]:
user_mat = filtered_users_training.drop(labels=['WindowID', 'Split', 'City', 'State', 'ZipCode', 'Country', 'GraduationDate'], axis=1)
binary_map = {'Yes': 1, 'No': 0}
user_mat['CurrentlyEmployed'] = user_mat['CurrentlyEmployed'].map(binary_map)
user_mat['ManagedOthers'] = user_mat['ManagedOthers'].map(binary_map)

In [10]:
filtered_users_training['DegreeType'] = filtered_users_training['DegreeType'].fillna('')
filtered_users_training['Major'] = filtered_users_training['Major'].fillna('')

filtered_users_training['DegreeType'] = filtered_users_training['DegreeType'] + filtered_users_training['Major']

/tmp/ipykernel_32/1636531677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_users_training['DegreeType'] = filtered_users_training['DegreeType'].fillna('')
/tmp/ipykernel_32/1636531677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_users_training['Major'] = filtered_users_training['Major'].fillna('')
/tmp/ipykernel_32/1636531677.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix1 = tf.fit_transform(filtered_users_training['DegreeType'])

In [13]:
tfidf_matrix1_T = tfidf_matrix1.T
cosine_sim1 = cosine_similarity(tfidf_matrix1)
cosine_sim1

array([[1.        , 0.        , 0.14288811, ..., 0.06168868, 0.14288811,
        1.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.14288811, 0.        , 1.        , ..., 0.07102979, 1.        ,
        0.14288811],
       ...,
       [0.06168868, 0.        , 0.07102979, ..., 1.        , 0.07102979,
        0.06168868],
       [0.14288811, 0.        , 1.        , ..., 0.07102979, 1.        ,
        0.14288811],
       [1.        , 0.        , 0.14288811, ..., 0.06168868, 0.14288811,
        1.        ]])

In [14]:
#from sklearn.metrics.pairwise import cosine_similarity
num_user_mat = user_mat.drop(labels=['UserID', 'DegreeType', 'Major'], axis=1)
num_user_mat = num_user_mat.fillna(0)
#numeric_features = np.array(num_user_mat).reshape(-1, 1)
#numeric_cosine_sim = cosine_similarity(numeric_features)

#def cosine_similarity(user_item_matrix):
    #similarity_matrix = np.dot(user_item_matrix, user_item_matrix.T) / (np.linalg.norm(user_item_matrix, axis=1)[:, np.newaxis] * np.linalg.norm(user_item_matrix, axis=1))
    #return similarity_matrix

#numeric_cosine_sim = cosine_similarity(num_user_mat)

#num_user_mat_T = num_user_mat.T
#pearson_similarity_matrix = num_user_mat_T.corr(method='pearson')

#numeric_cosine_sim = pearson_similarity_matrix

In [15]:
numeric_cosine_sim = cosine_similarity(num_user_mat)

In [16]:
#numeric_cosine_sim = pearson_similarity_matrix
user_similarity = numeric_cosine_sim

In [21]:
user_similarity

,0,1,2,3,4,5,6,7,8,9,...,9740,9741,9742,9743,9744,9745,9746,9747,9748,9749
0,1.000000,0.235406,0.363028,0.281076,0.283495,0.283286,0.382489,0.229730,0.385789,0.283733,...,0.284410,0.380381,0.219851,0.283769,0.285011,0.271998,0.982641,0.316062,0.321173,0.988610
1,0.235406,1.000000,0.235062,0.202642,0.220598,0.287528,0.211545,0.334684,0.251932,0.250474,...,0.234996,0.265577,0.172028,0.244713,0.245639,0.470197,0.276656,0.271438,0.283776,0.268782
2,0.363028,0.235062,1.000000,0.250357,0.256400,0.262194,0.979215,0.230047,0.977292,0.262981,...,0.260218,0.976323,0.268440,0.261869,0.262619,0.259063,0.359277,0.309799,0.937201,0.362254
3,0.281076,0.202642,0.250357,1.000000,0.310675,0.273711,0.281600,0.196814,0.273553,0.302950,...,0.279153,0.262500,0.218823,0.305242,0.293788,0.252243,0.246524,0.252243,0.182659,0.255970
4,0.283495,0.220598,0.256400,0.310675,1.000000,0.281624,0.283565,0.211330,0.277980,0.335144,...,0.284699,0.270194,0.221437,0.338174,0.311706,0.265659,0.258130,0.265659,0.204779,0.268636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745,0.271998,0.470197,0.259063,0.252243,0.265659,0.422275,0.259347,0.447988,0.278547,0.280191,...,0.273574,0.282870,0.206511,0.278240,0.276354,1.000000,0.283859,0.285714,0.263736,0.285591
9746,0.982641,0.276656,0.359277,0.246524,0.258130,0.274521,0.353363,0.273200,0.379655,0.276438,...,0.267249,0.385518,0.200758,0.272850,0.274305,0.283859,1.000000,0.327922,0.370152,0.997558
9747,0.316062,0.271438,0.309799,0.252243,0.265659,0.279586,0.310082,0.264194,0.329282,0.280191,...,0.273574,0.333605,0.206511,0.278240,0.276354,0.285714,0.327922,1.000000,0.314472,0.329654
9748,0.321173,0.283776,0.937201,0.182659,0.204779,0.235441,0.909350,0.279735,0.951566,0.237553,...,0.220837,0.967893,0.159880,0.231867,0.230295,0.263736,0.370152,0.314472,1.000000,0.362455


In [17]:
cosine_sim2 = pd.DataFrame(cosine_sim1)
cosine_sim2

,0,1,2,3,4,5,6,7,8,9,...,9740,9741,9742,9743,9744,9745,9746,9747,9748,9749
0,1.000000,0.000000,0.142888,0.000000,0.000000,0.000000,0.142888,0.000000,0.142888,0.000000,...,0.0,0.142888,0.0,0.000000,0.000000,0.000000,1.000000,0.061689,0.142888,1.000000
1,0.000000,1.000000,0.000000,0.000000,0.000000,0.055587,0.000000,0.071600,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.278262,0.000000,0.000000,0.000000,0.000000
2,0.142888,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.142888,0.071030,1.000000,0.142888
3,0.000000,0.000000,0.000000,1.000000,0.039365,0.000000,0.000000,0.000000,0.000000,0.041082,...,0.0,0.000000,0.0,0.041931,0.023405,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.039365,1.000000,0.000000,0.000000,0.000000,0.000000,0.076223,...,0.0,0.000000,0.0,0.077797,0.043424,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745,0.000000,0.278262,0.000000,0.000000,0.000000,0.199765,0.000000,0.257312,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
9746,1.000000,0.000000,0.142888,0.000000,0.000000,0.000000,0.142888,0.000000,0.142888,0.000000,...,0.0,0.142888,0.0,0.000000,0.000000,0.000000,1.000000,0.061689,0.142888,1.000000
9747,0.061689,0.000000,0.071030,0.000000,0.000000,0.000000,0.071030,0.000000,0.071030,0.000000,...,0.0,0.071030,0.0,0.000000,0.000000,0.000000,0.061689,1.000000,0.071030,0.061689
9748,0.142888,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.142888,0.071030,1.000000,0.142888


In [19]:
user_similarity = (user_similarity*2 + cosine_sim1*5)/7

In [20]:
user_similarity = pd.DataFrame(user_similarity)
user_similarity

,0,1,2,3,4,5,6,7,8,9,...,9740,9741,9742,9743,9744,9745,9746,9747,9748,9749
0,1.000000,0.235406,0.363028,0.281076,0.283495,0.283286,0.382489,0.229730,0.385789,0.283733,...,0.284410,0.380381,0.219851,0.283769,0.285011,0.271998,0.982641,0.316062,0.321173,0.988610
1,0.235406,1.000000,0.235062,0.202642,0.220598,0.287528,0.211545,0.334684,0.251932,0.250474,...,0.234996,0.265577,0.172028,0.244713,0.245639,0.470197,0.276656,0.271438,0.283776,0.268782
2,0.363028,0.235062,1.000000,0.250357,0.256400,0.262194,0.979215,0.230047,0.977292,0.262981,...,0.260218,0.976323,0.268440,0.261869,0.262619,0.259063,0.359277,0.309799,0.937201,0.362254
3,0.281076,0.202642,0.250357,1.000000,0.310675,0.273711,0.281600,0.196814,0.273553,0.302950,...,0.279153,0.262500,0.218823,0.305242,0.293788,0.252243,0.246524,0.252243,0.182659,0.255970
4,0.283495,0.220598,0.256400,0.310675,1.000000,0.281624,0.283565,0.211330,0.277980,0.335144,...,0.284699,0.270194,0.221437,0.338174,0.311706,0.265659,0.258130,0.265659,0.204779,0.268636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745,0.271998,0.470197,0.259063,0.252243,0.265659,0.422275,0.259347,0.447988,0.278547,0.280191,...,0.273574,0.282870,0.206511,0.278240,0.276354,1.000000,0.283859,0.285714,0.263736,0.285591
9746,0.982641,0.276656,0.359277,0.246524,0.258130,0.274521,0.353363,0.273200,0.379655,0.276438,...,0.267249,0.385518,0.200758,0.272850,0.274305,0.283859,1.000000,0.327922,0.370152,0.997558
9747,0.316062,0.271438,0.309799,0.252243,0.265659,0.279586,0.310082,0.264194,0.329282,0.280191,...,0.273574,0.333605,0.206511,0.278240,0.276354,0.285714,0.327922,1.000000,0.314472,0.329654
9748,0.321173,0.283776,0.937201,0.182659,0.204779,0.235441,0.909350,0.279735,0.951566,0.237553,...,0.220837,0.967893,0.159880,0.231867,0.230295,0.263736,0.370152,0.314472,1.000000,0.362455


In [63]:
num_user_mat.reset_index(drop=True).head()

,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,3,10.0,1.0,0,0
1,10,8.0,1.0,0,0
2,5,11.0,1.0,1,5
3,1,9.0,1.0,0,0
4,3,14.0,0.0,0,0


## Jobs-based collaborative filtering model

In [3]:
jobs = pd.read_csv('/kaggle/temp/jobs.tsv', delimiter='\t',encoding='utf-8', nrows=99999)
jobs = jobs.loc[jobs['Country']=='US']
jobs = jobs[jobs['JobID'] < 50000]
jobs['Title'] = jobs['Title'].fillna('')
jobs['Description'] = jobs['Description'].fillna('')
jobs['Requirements'] = jobs['Requirements'].fillna('')
jobs['Description'] = jobs['Title'] + jobs['Description']

/tmp/ipykernel_32/801242626.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv('/kaggle/temp/jobs.tsv', delimiter='\t',encoding='utf-8', nrows=99999)


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(jobs['Description'])

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix)

In [11]:
pd.DataFrame(cosine_sim)

,0,1,2,3,4,5,6,7,8,9,...,13281,13282,13283,13284,13285,13286,13287,13288,13289,13290
0,1.000000,0.039956,0.005888,0.004132,0.005422,0.002853,0.008027,0.000000,0.001695,0.000775,...,0.025436,0.015632,0.086193,0.058157,0.103602,0.014956,0.015215,0.066403,0.015717,0.015273
1,0.039956,1.000000,0.005335,0.019869,0.005669,0.007520,0.001229,0.000904,0.000910,0.004926,...,0.013449,0.023869,0.023282,0.016574,0.032770,0.032318,0.008498,0.011914,0.029988,0.004034
2,0.005888,0.005335,1.000000,0.048836,0.031260,0.037096,0.063756,0.073479,0.044957,0.036226,...,0.002667,0.011888,0.005823,0.006007,0.012769,0.005596,0.001917,0.000000,0.008552,0.004989
3,0.004132,0.019869,0.048836,1.000000,0.028400,0.023533,0.045195,0.029279,0.031906,0.031732,...,0.002148,0.011741,0.006391,0.003812,0.004507,0.005544,0.003270,0.001057,0.019631,0.001582
4,0.005422,0.005669,0.031260,0.028400,1.000000,0.013953,0.025149,0.018486,0.015954,0.016138,...,0.004303,0.004211,0.004485,0.006245,0.001712,0.002491,0.000000,0.000000,0.004595,0.001200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13286,0.014956,0.032318,0.005596,0.005544,0.002491,0.003765,0.006702,0.004941,0.005089,0.003993,...,0.045472,0.094236,0.058570,0.028314,0.025617,1.000000,0.063295,0.022524,0.012971,0.005221
13287,0.015215,0.008498,0.001917,0.003270,0.000000,0.002621,0.000000,0.000000,0.000000,0.002974,...,0.079643,0.093423,0.141316,0.126569,0.086189,0.063295,1.000000,0.003827,0.002354,0.122015
13288,0.066403,0.011914,0.000000,0.001057,0.000000,0.001017,0.000000,0.001278,0.001542,0.004580,...,0.005349,0.006429,0.052976,0.053464,0.110978,0.022524,0.003827,1.000000,0.006520,0.008610
13289,0.015717,0.029988,0.008552,0.019631,0.004595,0.003619,0.011406,0.005301,0.006716,0.004020,...,0.002603,0.021712,0.025072,0.005920,0.026460,0.012971,0.002354,0.006520,1.000000,0.024592


In [13]:
jobs['Description'][1]

'SAP Business Analyst / WM<strong>NO Corp. to Corp resumes&nbsp;are being considered for this &ldquo;Direct Hire" permanent position</strong>\\r<p><strong>H-1B Transfers ARE Available for Qualified Candidates with 5+ years of WM experience<br />\\r</strong>&nbsp;</p>\\r<p><b>SAP/ WM Warehouse Management Sr. Business Analyst</b></p>\\r<p>Position&rsquo;s Location is the Charlotte, NC metro area<br />\\r<br />\\r</p>\\r<p><i>We are currently seeking a Senior SAP Business Analyst with a minimum of 5 years of SAP/ WM Warehouse Management experience for a "<b>direct hire opportunity"</b> in the Charlotte, NC area. The successful candidate will have expertise providing requirements development, hands-on day-to-day support of internal customers and relaying their specifications to technical analysts. In this role you will develop best business practice knowledge and support the business in integrating this into SAP solution. </i><i></i></p>\\r<p><b></b>&nbsp;</p>\\r<p><b>You will serve as &he

## Applicants-Jobs One-hot Matrix

In [22]:
filtered_apps_training.head()

,UserID,WindowID,Split,ApplicationDate,JobID
0,47,1,Train,2012-04-04 15:56:23.537,169528
1,47,1,Train,2012-04-06 01:03:00.003,284009
2,47,1,Train,2012-04-05 02:40:27.753,2121
3,47,1,Train,2012-04-05 02:37:02.673,848187
4,47,1,Train,2012-04-05 22:44:06.653,733748


In [23]:
user_item_matrix = pd.crosstab(filtered_apps_training['JobID'], filtered_apps_training['UserID'])

In [24]:
user_item_matrix = user_item_matrix.T
user_item_matrix = user_item_matrix.rename_axis('', axis=1)

## Applicant-based Recommendation

In [25]:
df = pd.DataFrame(user_similarity)
#df.index = user_mat.UserID  
user_similarity = df
user_similarity = user_similarity.reset_index(drop=True)

user_item_matrix = user_item_matrix.reset_index(drop=True)

In [26]:
new_columns = {col: idx for idx, col in enumerate(user_similarity.columns)}
user_similarity.rename(columns=new_columns, inplace=True)

In [27]:
user_item_matrix

,6,11,18,40,62,89,107,115,141,142,...,1116069,1116075,1116077,1116085,1116103,1116121,1116127,1116142,1116144,1116286
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9746,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
user_similarity

,0,1,2,3,4,5,6,7,8,9,...,9740,9741,9742,9743,9744,9745,9746,9747,9748,9749
0,1.000000,0.235406,0.363028,0.281076,0.283495,0.283286,0.382489,0.229730,0.385789,0.283733,...,0.284410,0.380381,0.219851,0.283769,0.285011,0.271998,0.982641,0.316062,0.321173,0.988610
1,0.235406,1.000000,0.235062,0.202642,0.220598,0.287528,0.211545,0.334684,0.251932,0.250474,...,0.234996,0.265577,0.172028,0.244713,0.245639,0.470197,0.276656,0.271438,0.283776,0.268782
2,0.363028,0.235062,1.000000,0.250357,0.256400,0.262194,0.979215,0.230047,0.977292,0.262981,...,0.260218,0.976323,0.268440,0.261869,0.262619,0.259063,0.359277,0.309799,0.937201,0.362254
3,0.281076,0.202642,0.250357,1.000000,0.310675,0.273711,0.281600,0.196814,0.273553,0.302950,...,0.279153,0.262500,0.218823,0.305242,0.293788,0.252243,0.246524,0.252243,0.182659,0.255970
4,0.283495,0.220598,0.256400,0.310675,1.000000,0.281624,0.283565,0.211330,0.277980,0.335144,...,0.284699,0.270194,0.221437,0.338174,0.311706,0.265659,0.258130,0.265659,0.204779,0.268636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745,0.271998,0.470197,0.259063,0.252243,0.265659,0.422275,0.259347,0.447988,0.278547,0.280191,...,0.273574,0.282870,0.206511,0.278240,0.276354,1.000000,0.283859,0.285714,0.263736,0.285591
9746,0.982641,0.276656,0.359277,0.246524,0.258130,0.274521,0.353363,0.273200,0.379655,0.276438,...,0.267249,0.385518,0.200758,0.272850,0.274305,0.283859,1.000000,0.327922,0.370152,0.997558
9747,0.316062,0.271438,0.309799,0.252243,0.265659,0.279586,0.310082,0.264194,0.329282,0.280191,...,0.273574,0.333605,0.206511,0.278240,0.276354,0.285714,0.327922,1.000000,0.314472,0.329654
9748,0.321173,0.283776,0.937201,0.182659,0.204779,0.235441,0.909350,0.279735,0.951566,0.237553,...,0.220837,0.967893,0.159880,0.231867,0.230295,0.263736,0.370152,0.314472,1.000000,0.362455


In [29]:
def user_based_recommendation(user_item_matrix, user_similarity, user_id, top_n=5, min_interest=0.5):
    # Get the rating record of user user_id
    user_ratings = user_item_matrix.loc[user_id]

    # Find the top_n users most similar to user_id
    most_similar_users = user_similarity[user_id].sort_values(ascending=False)[1:top_n+1]

    recommendations = []

    # Traverse similar users
    for similar_user_id, similarity_score in most_similar_users.items():
        # Find jobs that similar users have applied for but the current user has not applied for
        new_items = user_item_matrix.loc[similar_user_id][user_ratings == 0]

        # Calculate recommendation score (pearson corr)
        recommendation_score = similarity_score

        # Add items with similarity greater than 0.5 to the recommendation list
        for item, interest in new_items.items():
            if interest > min_interest:
                recommendations.append((item, recommendation_score))

    # Sort in descending order according to the recommendation score, and select the top_n items as the recommendation results
    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = recommendations[:top_n]

    return top_recommendations

top_recommendations = user_based_recommendation(user_item_matrix, user_similarity, 15)
top_top_recommendations = pd.DataFrame(top_recommendations)
print("Recommendations for User", 15, ":")
top_recommendations

Recommendations for User 15 :


[(879, 0.6829213900259018),
 (42895, 0.6829213900259018),
 (274809, 0.6829213900259018),
 (972094, 0.5672676431398841),
 (460020, 0.559696514677029)]

In [30]:
user_ratings = user_item_matrix.loc[15]
most_similar_users = user_similarity[15].sort_values(ascending=False)[1:6]

In [31]:
most_similar_users

4854    0.682921
8617    0.567268
7699    0.559697
1899    0.557715
4556    0.487251
Name: 15, dtype: float64

In [54]:
print(user_similarity.iloc[15, 4854])
print(user_similarity.iloc[15, 8617])
print(user_similarity.iloc[15, 7699])
print(user_similarity.iloc[15, 1899])
print(user_similarity.iloc[15, 4556])

0.6829213900259018
0.5672676431398841
0.559696514677029
0.5577149662713068
0.48725094114107403
